In [1]:
import os,sys
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
import shutil
import time
import importlib

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import modules
from bivme.preprocessing.dicom.select_views import select_views
from bivme.preprocessing.dicom.segment_views import segment_views
from bivme.preprocessing.dicom.generate_contours import generate_contours
from bivme.preprocessing.dicom.export_guidepoints import export_guidepoints

In [3]:
# Check if GPU is available (torch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


## This code reads in DICOM files and generates GPFiles for personalised biventricular mesh fitting.

## Step 0: Set up directories
The preprocessing pipeline expects DICOM files to be organised in a certain way. There are three key things to get right with the DICOMs before running the code. 

Firstly, there should be no non-cine images or non-cardiac views. My job is hard enough as it is. Some day I'll add a pre-preprocessing step, but not today.

Secondly, images should be separated into folders by case, like so:

    input_path   
    └─── case1
        │─── *.dcm
    └─── case2
        │─── *.dcm
    └─── ...
    
Thirdly, all images should be converted to .dcm format.

In [7]:
batch_ID = 'test' # This will serve as your output folder name. Example: 'test'
analyst_id = 'analyst1' # Example: 'analyst1'
input_path = '' # Path to the input DICOM folder
processed_path = '' # Path to the processed folder, where view predictions and segmentations will be stored. This will be created upon run time.
states_path = '' # Path to the states folder, where the logs and view predictions will be stored for reference. This will be created upon run time.
output_path = '' # Path to the output folder, where GP files will be stored. This will be created upon run time.

# Target path: src/bivme/preprocessing/dicom/models
cwd = os.getcwd()
print('Current working directory:', cwd)
MODEL_DIR = os.path.join(cwd,'models')

Current working directory: c:\Users\jdil469\Code\biv-me\src\bivme\preprocessing\dicom


## Step 0.5: Choose case

In [8]:
case = '' # Enter the case ID. This should be a subfolder within the input_path. 

In [9]:
src = os.path.join(input_path)
dst = os.path.join(processed_path, batch_ID)
states = os.path.join(states_path, batch_ID)
output = os.path.join(output_path, batch_ID)

os.makedirs(dst, exist_ok=True)
os.makedirs(states, exist_ok=True)
os.makedirs(output, exist_ok=True)

case_src = os.path.join(src, case)
if not os.path.isdir(case_src):
    print(f'Case {case} not found in source folder. Please check the case ID.')
    sys.exit()

case_dst = os.path.join(dst, case)

if os.path.exists(case_dst):
    enter = input(f'Case {case} already processed. Do you want to overwrite? (y/n): ')
    if enter == 'y':
        shutil.rmtree(case_dst)
    else:
        print(f'Change the case ID or delete the existing folder {case_dst} before proceeding.')

In [10]:
start_time = time.time()
# Create log file to record some details
states = os.path.join(states, case, analyst_id)
os.makedirs(states, exist_ok=True)
if not os.path.exists(os.path.join(states, 'logs')):
    os.mkdir(os.path.join(states, 'logs'))

log_name = f'{case}_{analyst_id}_{time.ctime().replace(" ","-").replace(":","-")}.txt'
log_path = os.path.join(states, 'logs', log_name)
with open(log_path, 'w') as f:
    f.write(f'Case: {case}\n')
    f.write(f'Batch ID: {batch_ID}\n')
    f.write(f'Source: {case_src}\n')
    f.write(f'Destination: {dst}\n')
    f.write(f'Analyst ID: {analyst_id}\n')
    f.write(f'Processing started at: {time.ctime()}\n')
    f.write('\n')

print(f'Processing case: {case}')

Processing case: cardiohance_001


In [11]:
# Reload modules to ensure any changes are reflected
importlib.reload(sys.modules[select_views.__module__])
from select_views import select_views

## Step 1: View classification

In [12]:
option = 'default' # Either 'default' or 'load'. Default will automatically perform view prediction. Load will load the view predictions from the states folder.

slice_info_df, num_phases, slice_mapping = select_views(case, case_src, case_dst, MODEL_DIR, states, option)
print('View selection complete.')

print(f'Number of phases: {num_phases}')

Data prepared for view prediction. 14 image series found.
Running view predictions...
No duplicate slice locations found.
Multiple series classed as 3ch.
Excluded series [15]
View predictions for cardiohance_001:
SAX-atria: 1 series
SAX: 5 series
4ch: 1 series
3ch: 1 series
Excluded: 1 series
2ch: 1 series
2ch-RT: 1 series
LVOT: 1 series
RVOT: 1 series
RVOT-T: 1 series
View selection complete.
Number of phases: 27


## Manually correct view predictions (optional)
No view prediction network is perfect. You can manually correct the view predictions if needed. The view predictions will be saved in:

    states_path   
    └───batch_ID
        └───case
            └───analyst_id
                |───view_predictions.csv

You can visually inspect which images have been classified as which view (or excluded) here:


    processed_path   
    └───batch_ID
        └───case
            └───view-classsification
                └───sorted

If you do correct the views, make run the cell block below with the variable 'option' set to 'load'. 

In [ ]:
option = 'load' # Either 'default' or 'load'. Default will automatically perform view prediction. Load will load the view predictions from the states folder.

slice_info_df, num_phases, slice_mapping = select_views(case, case_src, case_dst, MODEL_DIR, states, option)
print('View selection complete.')

print(f'Number of phases: {num_phases}')

In [13]:
# Reload modules to ensure any changes are reflected
importlib.reload(sys.modules[segment_views.__module__])
from segment_views import segment_views

## Step 2: Segmentation

In [14]:
## Segmentation
version = '3d' # '2d' or '3d' segmentation models. 3D is recommended for better efficiency and temporal coherence across frames.
start_time = time.time()
segment_views(case, case_dst, MODEL_DIR, slice_info_df, version = version)
end_time = time.time()
print(f'Segmentation complete. Time taken: {end_time-start_time} seconds ({version} version).')

# Add segmentation time to log
with open(log_path, 'a') as f:
    f.write(f'Segmentation time: {end_time-start_time} seconds ({version} version).\n')


Writing SAX images to nifti files...

Segmenting SAX images...

*** Making predictions for SAX images ***
There are 5 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 5 cases that I would like to predict

Predicting SAX_3d_10:
perform_everything_on_device: True


100%|██████████| 1/1 [00:03<00:00,  3.69s/it]


sending off prediction to background worker for resampling and export
done with SAX_3d_10

Predicting SAX_3d_11:
perform_everything_on_device: True


100%|██████████| 1/1 [00:00<00:00,  1.98it/s]


sending off prediction to background worker for resampling and export
done with SAX_3d_11

Predicting SAX_3d_12:
perform_everything_on_device: True


...

sending off prediction to background worker for resampling and export
done with 3ch_3d_14
Done with 3ch

Writing 4ch images to nifti files...

Segmenting 4ch images...

*** Making predictions for 4ch images ***
There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting 4ch_3d_13:
perform_everything_on_device: True


100%|██████████| 1/1 [00:02<00:00,  2.69s/it]


sending off prediction to background worker for resampling and export
done with 4ch_3d_13
Done with 4ch

Writing RVOT images to nifti files...

Segmenting RVOT images...

*** Making predictions for RVOT images ***
There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting RVOT_3d_19:
perform_everything_on_device: True


100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


sending off prediction to background worker for resampling and export
done with RVOT_3d_19
Done with RVOT

Segmentation complete. Time taken: 95.30491971969604 seconds (3d version).


## Step 2.5: Review segmentations (optional)
Images and segmentations are stored here in nifti format (.nii.gz). 

    processed_path   
    └───batch_ID
        └───case


Hopefully it won't be necessary 99% of the time, but, if you wish, segmentations can be corrected here, and the rest of the code will incorporate those changes. 

By default, the contouring code (after segmentation) carries out some basic QC. All label types (except for the RV myocardium) have all but their largest components removed, so you shouldn't need to worry about fixing minor things such as removing extraneous label regions. If you find a problem with the exported contours or fitted models, it's more likely due to poor RV myo segmentation or poor segmentation around valve planes.   

3D Slicer or ITKSnap are good tools correcting segmentations. If you do, make sure to only change the segmentations with _2d_ in the name. These are the ones that get loaded later on for contouring & exporting. Both versions of segmentation model will output these _2d_ segmentation files. Unfortunately, this also means correcting segmentations frame by frame.

In [320]:
# Reload modules to ensure any changes are reflected
importlib.reload(sys.modules[generate_contours.__module__])
from generate_contours import generate_contours

## Step 3: Generate contours

In [16]:
slice_dict = generate_contours(case, case_dst, slice_info_df, num_phases, version = version)
print('Guide points generated.')

Generating contours for SAX slice 8...

Generating contours for SAX slice 9...

Generating contours for SAX slice 10...

Generating contours for SAX slice 11...

Generating contours for SAX slice 12...

Generating contours for 2ch slice 16...

Generating contours for 3ch slice 14...

Generating contours for 4ch slice 13...

Generating contours for RVOT slice 19...

Guide points generated.


In [17]:
# Reload to ensure any changes are reflected
importlib.reload(sys.modules[export_guidepoints.__module__])
from export_guidepoints import export_guidepoints

## Step 4: Export guidepoints

In [18]:
export_guidepoints(case, case_dst, output, slice_dict, slice_mapping)
print('Export complete.')
print(f'Case {case} complete.')
print(f'Total time taken: {time.time()-start_time} seconds.')

Export complete.
Case cardiohance_001 complete.
Total time taken: 102.80258440971375 seconds.
